# Advanced SchoolHoliday Transformations

SchoolHolidays seem to be erroneous for some stores: during summer holidays (and others) some stores only have schoolholiday=1 during the week and 0 on weekends. intuitively this feels wrong an should not benefit the model. so this notebook (and now the harmonize_school_holidays function in data_utils) find weekends during longer holidays to set schoolholiday=1 also during these weekends for all stores.

also add_column_for_last_holiday_week() marks the last week of summer holidays for each store. intuitively these two changes should make a difference and it indeed does marginally for CV score and improved LB score by >1%point.

In [1]:
from time import time
import datetime
from operator import itemgetter
import csv

import utils
import data_utils

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation as cv
from sklearn.grid_search import RandomizedSearchCV

from scipy.stats import randint as sp_randint

In [2]:
test = pd.read_csv('../../data/test.csv')
test.Date = test.Date.astype(np.datetime64)
test = test.sort_values('Date').reset_index(drop=True)

In [ ]:
from pandas.tseries.offsets import DateOffset

compare_offset = 1
dow = 6

oneday = DateOffset(days=compare_offset)
tmp = test[['Store']]
tmp['Date'] = test.Date + oneday
tmp['SchoolHoliday_m1'] = test.SchoolHoliday
tmp = pd.merge(test, tmp, how='left', on=['Store', 'Date'])
tmp['SchoolHoliday_m1'] = tmp.SchoolHoliday_m1.fillna(0).astype(int)
tmp['SchoolHoliday_m1'] = np.logical_and(tmp.DayOfWeek==dow, tmp.SchoolHoliday_m1==1)
tmp['SchoolHoliday'] = np.where(np.logical_or(tmp.SchoolHoliday==1, tmp.SchoolHoliday_m1), 1, 0)
tmp.drop('SchoolHoliday_m1', axis=1)[tmp.Store==600]
#test[tmp.Store==600]

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(test[test.Store==600].SchoolHoliday, alpha=0.5)
plt.plot(test[test.Store==600].Open, alpha=0.5)
plt.plot(test[test.Store==600].DayOfWeek, alpha=0.5)
#plt.plot(test[test.Store==600].Sales, alpha=0.5)